In [1]:
from pycalphad import Database, Model, calculate, equilibrium, variables as v
from xarray import DataArray

In [2]:
class PrecipitateModel(Model):
    matrix_chempots = []
    @property
    def matrix_hyperplane(self):
        return sum(self.moles(self.nonvacant_elements[i])*self.matrix_chempots[i]
                   for i in range(len(self.nonvacant_elements)))
    @property
    def GM(self):
        return self.ast - self.matrix_hyperplane

class GibbsThompsonModel(Model):
    "Spherical particle."
    radius = 1e-6 # m
    volume = 7.3e-6 # m^3/mol
    interfacial_energy = 250e-3 # J/m^2
    elastic_misfit_energy = 0 # J/mol
    @property
    def GM(self):
        return self.ast + (2*self.interfacial_energy/self.radius + self.elastic_misfit_energy) * self.volume

In [3]:
import numpy as np

def parallel_tangent(dbf, comps, matrix_phase, matrix_comp, precipitate_phase, temp):
    conds = {v.N: 1, v.P: 1e5, v.T: temp}
    conds.update(matrix_comp)
    matrix_eq = equilibrium(dbf, comps, matrix_phase, conds)
    # pycalphad currently doesn't have a way to turn global minimization off and directly specify starting points
    if matrix_eq.isel(vertex=1).Phase.values.flatten() != ['']:
        raise ValueError('Matrix phase has miscibility gap. This bug will be fixed in the future')
    matrix_chempots = matrix_eq.MU.values.flatten()
    # This part will not work until mass balance constraint can be relaxed
    #precip = PrecipitateModel(dbf, comps, precipitate_phase)
    #precip.matrix_chempots = matrix_chempots
    #conds = {v.N: 1, v.P: 1e5, v.T: temp}
    #df_eq = equilibrium(dbf, comps, precipitate_phase, conds, model=precip)
    df_eq = calculate(dbf, comps, precipitate_phase, T=temp, N=1, P=1e5)
    df_eq['GM'] = df_eq.X.values[0,0,0].dot(matrix_chempots) - df_eq.GM
    selected_idx = df_eq.GM.argmax()
    return matrix_eq.isel(vertex=0), df_eq.isel(points=selected_idx)

def nucleation_barrier(dbf, comps, matrix_phase, matrix_comp, precipitate_phase, temp,
                       interfacial_energy, precipitate_volume):
    "Spherical precipitate."
    matrix_eq, precip_eq = parallel_tangent(dbf, comps, matrix_phase, matrix_comp, precipitate_phase, temp)
    precip_driving_force = float(precip_eq.GM.values) # J/mol
    elastic_misfit_energy = 0 # J/m^3
    barrier = 16./3 * np.pi * interfacial_energy **3 / (precip_driving_force + elastic_misfit_energy) ** 2 # J/mol
    critical_radius = 2 * interfacial_energy / ((precip_driving_force / precipitate_volume) + elastic_misfit_energy) # m
    print(temp, critical_radius)
    if critical_radius < 0:
        barrier = np.inf
    cluster_area = 4*np.pi*critical_radius**2
    cluster_volume = (4./3) * np.pi * critical_radius**3
    return barrier, precip_driving_force, cluster_area, cluster_volume, matrix_eq, precip_eq

def growth_rate(dbf, comps, matrix_phase, matrix_comp, precipitate_phase, temp, particle_radius):
    conds = {v.N: 1, v.P: 1e5, v.T: temp}
    conds.update(matrix_comp)
    # Fictive; Could retrieve from TDB in principle
    mobility = 1e-7 * np.exp(-14e4/(8.3145*temp)) # m^2 / s
    mobilities = np.eye(len(matrix_comp)+1) * mobility

    matrix_ff_eq, precip_eq = parallel_tangent(dbf, comps, matrix_phase, matrix_comp, precipitate_phase, temp)
    precip_driving_force = float(precip_eq.GM.values) # J/mol
    interfacial_energy = 250e-3 # J/m^2
    precipitate_volume = 7.3e-6 # m^3/mol
    elastic_misfit_energy = 0 # J/m^3
    # Spherical particle
    critical_radius = 2 * interfacial_energy / ((precip_driving_force / precipitate_volume) + elastic_misfit_energy) # m
    # XXX: Should really be done with global min off, fixed-phase conditions, etc.
    # As written, this will break with miscibility gaps
    particle_mod = GibbsThompsonModel(dbf, comps, precipitate_phase)
    particle_mod.radius = particle_radius
    interface_eq = equilibrium(dbf, comps, [matrix_phase, precipitate_phase], conds,
                               model={precipitate_phase: particle_mod})
    matrix_idx = np.nonzero((interface_eq.Phase==matrix_phase).values.flatten())[0]
    if len(matrix_idx) > 1:
        raise ValueError('Matrix phase has miscibility gap')
    elif len(matrix_idx) == 0:
        # Matrix is metastable at this composition; massive transformation kinetics?
        print(interface_eq)
        raise ValueError('Matrix phase is not stable')
    else:
        matrix_idx = matrix_idx[0]
        matrix_interface_eq = interface_eq.isel(vertex=matrix_idx)

    precip_idx = np.nonzero((interface_eq.Phase==precipitate_phase).values.flatten())[0]
    if len(precip_idx) > 1:
        raise ValueError('Precipitate phase has miscibility gap')
    elif len(precip_idx) == 0:
        precip_conc = np.zeros(len(interface_eq.component))
        # Precipitate is metastable at this radius (it will start to dissolve)
        # Compute equilibrium for precipitate by itself at parallel tangent composition
        pt_comp = {v.X(str(comp)): precip_eq.X.values.flatten()[idx] for idx, comp in enumerate(precip_eq.component.values[:-1])}
        conds = {v.N: 1, v.P: 1e5, v.T: temp}
        conds.update(pt_comp)
        precip_interface_eq = equilibrium(dbf, comps, precipitate_phase, conds,
                               model={precipitate_phase: particle_mod})
        precip_interface_eq = precip_interface_eq.isel(vertex=0)
    else:
        precip_idx = precip_idx[0]
        precip_interface_eq = interface_eq.isel(vertex=precip_idx)
    
    eta_geo_factor = 1.0
    # Growth equation from Eq. 12, M. Paliwal and I.-H Jung, Calphad, 2019
    velocity = 2*interfacial_energy * precipitate_volume * (1./critical_radius - 1./particle_radius)
    x_int_precip = precip_interface_eq.X.values.flatten()
    x_int_matrix = matrix_interface_eq.X.values.flatten()
    denominator = 0
    for i in range(mobilities.shape[0]-1):
        for j in range(mobilities.shape[1]-1):
            denominator += (x_int_precip[i] - x_int_matrix[i]) * (x_int_precip[j] - x_int_matrix[j]) / mobilities[i,j]
    velocity /= denominator
    return velocity



def precipitate_size_distribution(dbf, comps, matrix_phase, initial_matrix_comp, precipitate_phase, temp):
    interfacial_energy = 250e-3 # J/m^2
    precipitate_volume = 7.3e-6 # m^3/mol
    elastic_misfit_energy = 0 # J/m^3
    atomic_distance = 2e-10 # m
    n_prefactor = 1e18
    # Could retrieve from TDB in principle
    diffusivity = 1e-7 * np.exp(-14e4/(8.3145*temp)) # m^2 / s
    delta_t = 1e-3
    time_steps = int(1e4)
    precipitate_dist = np.zeros((time_steps, 100))
    size_classes = np.logspace(-10, -6, num=100)
    print('size_classes', size_classes)
    # Compositions at interface
    matrix_composition = np.zeros((time_steps, len(initial_matrix_comp)+1))
    precipitate_composition = np.zeros((time_steps, len(initial_matrix_comp)+1))

    g_star, g_nucl, area, volume, matrix_eq, precip_eq = nucleation_barrier(dbf, comps, matrix_phase,
                                                                            initial_matrix_comp, precipitate_phase,
                                                                            temp, interfacial_energy,
                                                                            precipitate_volume)
    matrix_composition[0, :] = matrix_eq.X.values.flatten()
    precipitate_composition[0, :] = precip_eq.X.values.flatten()
    old_critical_radius = 1e-10
    for step in range(time_steps):
        if step > 0:
            matrix_comp = {v.X(c): matrix_composition[step][comp_idx] for comp_idx, c in enumerate(matrix_eq.component.values[:-1])}
            g_star, g_nucl, area, volume, matrix_eq, precip_eq = nucleation_barrier(dbf, comps, matrix_phase,
                                                                                    matrix_comp, precipitate_phase,
                                                                                    temp, interfacial_energy,
                                                                                    precipitate_volume)
            matrix_composition[step, :] = matrix_eq.X.values.flatten()
            precipitate_composition[step, :] = precip_eq.X.values.flatten()

        print('Step ', step, 'Matrix Comp: ', matrix_composition[step], 'Precipitate Comp:', precipitate_composition[step])
        precip_driving_force = float(precip_eq.GM.values) # J/mol
        elastic_misfit_energy = 0 # J/m^3
        critical_radius = 2 * interfacial_energy / ((precip_driving_force / precipitate_volume) + elastic_misfit_energy) # m

        zeldovich = np.sqrt(interfacial_energy/(8.3145*temp)) * precipitate_volume / (2*np.pi*6.022e23 * critical_radius**2) # dim-less

        conc_diff = (precip_eq.X.values.flatten() - matrix_eq.X.values.flatten()) ** 2 /\
                    (matrix_eq.X.values.flatten() * diffusivity)
        beta_attachment_rate = (area / ((atomic_distance**4) * precipitate_volume)) / conc_diff.sum()  # s^-1
        incubation_time = 1 / (2*beta_attachment_rate*zeldovich) # s

        # Now we know the nucleation rate, and the critical radius
        # XXX: Missing the incubation contribution
        ss_nucl_rate = n_prefactor * zeldovich * beta_attachment_rate * np.exp(-g_star/(8.3145*temp))
        print('ss_nucl_rate', ss_nucl_rate)
        if critical_radius < 0:
            critical_radius = 0.0
        print('critical_radius', critical_radius)
        active_size_class_idx = 0
        for sc_idx in range(size_classes.shape[0]):
            if critical_radius > size_classes[sc_idx]:
                active_size_class_idx = sc_idx
        if critical_radius > size_classes[-1]:
            active_size_class_idx = size_classes.shape[0] - 1
        print('active_size_class_idx', active_size_class_idx)
        precipitate_dist[step, active_size_class_idx] += ss_nucl_rate * delta_t

        # Compute growth rates for all size classes
        velocities = np.zeros(precipitate_dist.shape[1])
        matrix_comp = {v.X(c): matrix_composition[step][comp_idx] for comp_idx, c in enumerate(matrix_eq.component.values[:-1])}
        for idx in range(1, size_classes.shape[0]):
            radius = size_classes[idx]
            velocities[idx-1] = growth_rate(dbf, comps, matrix_phase, matrix_comp, precipitate_phase, temp, radius)
        vel_sign = np.sign(velocities)
        vel_sign[vel_sign<0] = 0
        # Explicit finite difference method with first-order upwinding scheme
        for i in range(1, size_classes.shape[0]-1):
            delta_r = size_classes[i] - size_classes[i-1]
            if (velocities[i-1] > 0) and velocities[i] > 0: 
                precipitate_dist[step+1, i] = precipitate_dist[step, i] + (delta_t / delta_r) * velocities[i-1] * precipitate_dist[step, i-1] - (delta_t / delta_r) * velocities[i] * precipitate_dist[step, i]
            elif (velocities[i-1] > 0) and velocities[i] < 0: 
                precipitate_dist[step+1, i] = precipitate_dist[step, i] + (delta_t / delta_r) * velocities[i-1] * precipitate_dist[step, i-1] - (delta_t / delta_r) * velocities[i] * precipitate_dist[step, i+1]
            elif (velocities[i-1] < 0) and velocities[i] > 0: 
                precipitate_dist[step+1, i] = precipitate_dist[step, i] + (delta_t / delta_r) * velocities[i-1] * precipitate_dist[step, i] - (delta_t / delta_r) * velocities[i] * precipitate_dist[step, i]
            else:
                precipitate_dist[step+1, i] = precipitate_dist[step, i] + (delta_t / delta_r) * velocities[i-1] * precipitate_dist[step, i] - (delta_t / delta_r) * velocities[i] * precipitate_dist[step, i+1]
        print('Particle Conservation Error', (np.sum(precipitate_dist[step+1]) - np.sum(precipitate_dist[step]))/np.sum(precipitate_dist[step]))
        print('Critical Radius Change', (critical_radius - old_critical_radius)/old_critical_radius)
        old_critical_radius = critical_radius
        # Tiny particles dissolve
        precipitate_dist[step+1, 1] = 0
        # Update volume fraction of precipitate (spherical assumption)
        total_precipitate_volume = np.dot((4/3) * np.pi * size_classes ** 3, precipitate_dist[step+1,:]) # m^3
        print('precipitate_dist', precipitate_dist[step+1])
        print('total_precipitate_volume', total_precipitate_volume)
        matrix_domain_volume = precipitate_volume * 1 # because N=1; assume equal for simplicity; m^3
        precip_volume_fraction = total_precipitate_volume / matrix_domain_volume
        print('precip_volume_fraction', precip_volume_fraction)
        if step == time_steps - 1:
            break
        # Update matrix composition
        matrix_composition[step+1, :] = (matrix_composition[0 , :] - precip_volume_fraction * precipitate_composition[step]) / (1 - precip_volume_fraction)
    return matrix_composition, precipitate_composition, precipitate_dist

In [4]:
dbf = Database('Al-Cu-Zr_Zhou.tdb')
comps = ['AL', 'CU', 'VA']

results = precipitate_size_distribution(dbf, comps, 'BCC_A2', {v.X('CU'): 0.6}, 'ETA2', 800)
matrix_composition = results[0]
precipitate_composition = results[1]
precipitate_dist = results[2]

size_classes [1.00000000e-10 1.09749877e-10 1.20450354e-10 1.32194115e-10
 1.45082878e-10 1.59228279e-10 1.74752840e-10 1.91791026e-10
 2.10490414e-10 2.31012970e-10 2.53536449e-10 2.78255940e-10
 3.05385551e-10 3.35160265e-10 3.67837977e-10 4.03701726e-10
 4.43062146e-10 4.86260158e-10 5.33669923e-10 5.85702082e-10
 6.42807312e-10 7.05480231e-10 7.74263683e-10 8.49753436e-10
 9.32603347e-10 1.02353102e-09 1.12332403e-09 1.23284674e-09
 1.35304777e-09 1.48496826e-09 1.62975083e-09 1.78864953e-09
 1.96304065e-09 2.15443469e-09 2.36448941e-09 2.59502421e-09
 2.84803587e-09 3.12571585e-09 3.43046929e-09 3.76493581e-09
 4.13201240e-09 4.53487851e-09 4.97702356e-09 5.46227722e-09
 5.99484250e-09 6.57933225e-09 7.22080902e-09 7.92482898e-09
 8.69749003e-09 9.54548457e-09 1.04761575e-08 1.14975700e-08
 1.26185688e-08 1.38488637e-08 1.51991108e-08 1.66810054e-08
 1.83073828e-08 2.00923300e-08 2.20513074e-08 2.42012826e-08
 2.65608778e-08 2.91505306e-08 3.19926714e-08 3.51119173e-08
 3.85352859

Particle Conservation Error 8.31312046747168e-08
Critical Radius Change 1.1041730822064369e-06
precipitate_dist [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 5.11923817e-12 9.71141197e-07 1.13225288e-01
 1.12540972e+04 1.18230556e+09 1.72515908e+14 6.36054265e+08
 2.76267231e+03 9.17140408e-03 1.85766674e-08 1.66019216e-14
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.0

Particle Conservation Error 8.316917370560544e-08
Critical Radius Change 1.1077250435790689e-06
precipitate_dist [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.59651928e-32 1.28849153e-27 6.10738452e-23 2.27352693e-18
 7.51080281e-14 2.36501929e-09 7.47776757e-05 2.49100512e+00
 9.28496811e+04 4.33530170e+09 3.45027432e+14 2.33193851e+09
 2.27892291e+04 2.01745352e-01 1.43022115e-06 7.66910896e-12
 2.98031454e-17 8.01022030e-23 1.39307187e-28 1.39325164e-34
 6.01479867e-41 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.

Particle Conservation Error 8.320725164590743e-08
Critical Radius Change 1.1097509551770346e-06
precipitate_dist [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 2.59887647e-50
 9.56070171e-46 2.05541531e-41 3.44418348e-37 5.06333651e-33
 6.97193720e-29 9.37849969e-25 1.27016319e-20 1.77318221e-16
 2.60358930e-12 4.09924744e-08 7.06987043e-04 1.37385391e+01
 3.15137653e+05 9.45923592e+09 5.17534573e+14 5.08729732e+09
 7.73357034e+04 1.11251543e+00 1.35203690e-05 1.32914265e-10
 1.03304387e-15 6.24717457e-21 2.89721966e-26 1.01415766e-31
 2.62692818e-37 4.90152137e-43 6.33402759e-49 5.32306055e-55
 2.59014424e-61 5.48173493e-68 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.

Particle Conservation Error 8.324521877046932e-08
Critical Radius Change 1.1110408821032903e-06
precipitate_dist [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 9.75260454e-66 1.64482144e-61
 1.62177364e-57 1.24613827e-53 8.39304324e-50 5.28486329e-46
 3.24069094e-42 1.99077881e-38 1.25106060e-34 8.17482403e-31
 5.62849990e-27 4.13004590e-23 3.26300941e-19 2.80334496e-15
 2.64621729e-11 2.77766090e-07 3.29359907e-03 4.51795405e+01
 7.48479161e+05 1.65543556e+10 6.90037330e+14 8.90177542e+09
 1.83649590e+05 3.65800133e+00 6.29786985e-05 9.00541927e-10
 1.04988651e-14 9.87626551e-20 7.44292927e-25 4.46633318e-30
 2.12097336e-35 7.91494348e-41 2.30133193e-46 5.15738196e-52
 8.78334010e-58 1.11533367e-63 1.02824675e-69 6.61680883e-76
 2.79023382e-82 6.86280857e-89 7.

...

Particle Conservation Error 8.446193220943445e-08
Critical Radius Change 1.1115760491472776e-06
precipitate_dist [0.00000000e+000 0.00000000e+000 3.39949381e-062 3.39579911e-061
 3.90973850e-060 5.19630444e-059 7.98244232e-058 1.41882030e-056
 2.92038649e-055 6.96580536e-054 1.92642845e-052 6.17962605e-051
 2.30000810e-049 9.93447070e-048 4.98041582e-046 2.89816050e-044
 1.95760305e-042 1.53485064e-040 1.39678892e-038 1.47536020e-036
 1.80862641e-034 2.57319131e-032 4.24885658e-030 8.14287458e-028
 1.81156422e-025 4.67966669e-023 1.40425440e-020 4.89809432e-018
 1.98780662e-015 9.39923347e-013 5.18876985e-010 3.35418686e-007
 2.55039125e-004 2.29688907e-001 2.47802769e+002 3.26709337e+005
 5.47628070e+008 1.28586214e+012 6.20781170e+015 6.88055375e+011
 1.33681718e+008 2.63289490e+004 4.71950652e+000 7.42329692e-004
 1.00967547e-007 1.18046627e-011 1.18387877e-015 1.01826468e-019
 7.51703049e-024 4.76875458e-028 2.60361480e-032 1.22529637e-036
 4.97833409e-041 1.74895656e-045 5.3208070

Particle Conservation Error 8.449997104576979e-08
Critical Radius Change 1.1110943807112584e-06
precipitate_dist [0.00000000e+000 0.00000000e+000 1.02149942e-061 9.89621465e-061
 1.10477342e-059 1.42344278e-058 2.11956016e-057 3.65146002e-056
 7.28434779e-055 1.68395203e-053 4.51366009e-052 1.40338261e-050
 5.06303178e-049 2.11997414e-047 1.03038506e-045 5.81371097e-044
 3.80807992e-042 2.89570933e-040 2.55615407e-038 2.61929298e-036
 3.11550839e-034 4.30142091e-032 6.89348209e-030 1.28244513e-027
 2.76998612e-025 6.94816818e-023 2.02489111e-020 6.86044836e-018
 2.70481704e-015 1.24269159e-012 6.66670123e-010 4.18868460e-007
 3.09605591e-004 2.71094599e-001 2.84401947e+002 3.64669527e+005
 5.94565682e+008 1.35814489e+012 6.38016986e+015 7.26621630e+011
 1.45116447e+008 2.93838025e+004 5.41587768e+000 8.76062150e-004
 1.22561577e-007 1.47411007e-011 1.52109891e-015 1.34634292e-019
 1.02295805e-023 6.68044698e-028 3.75525734e-032 1.81987177e-036
 7.61544822e-041 2.75599394e-045 8.6385624

Particle Conservation Error 8.453801035364501e-08
Critical Radius Change 1.1113487956755945e-06
precipitate_dist [0.00000000e+000 0.00000000e+000 2.96935497e-061 2.79302713e-060
 3.02657313e-059 3.78447766e-058 5.46812865e-057 9.13998862e-056
 1.76901822e-054 3.96757527e-053 1.03176829e-051 3.11244853e-050
 1.08951596e-048 4.42672163e-047 2.08794125e-045 1.14336010e-043
 7.26933780e-042 5.36605366e-040 4.59889430e-038 4.57587777e-036
 5.28570315e-034 7.08810983e-032 1.10347924e-029 1.99450265e-027
 4.18608162e-025 1.02046736e-022 2.89063906e-020 9.52078874e-018
 3.64965150e-015 1.63055208e-012 8.50755304e-010 5.19945671e-007
 3.73886404e-004 3.18541363e-001 3.25203070e+002 4.05845716e+005
 6.44112706e+008 1.43240722e+012 6.55252364e+015 7.66234939e+011
 1.57184292e+008 3.26968588e+004 6.19208292e+000 1.02928851e-003
 1.47997973e-007 1.82976847e-011 1.94113064e-015 1.76665233e-019
 1.38044489e-023 9.27259177e-028 5.36215032e-032 2.67370701e-036
 1.15136528e-040 4.28856759e-045 1.3837714

Particle Conservation Error 8.457606007822641e-08
Critical Radius Change 1.0981643009201184e-06
precipitate_dist [0.00000000e+000 0.00000000e+000 8.36598740e-061 7.64822157e-060
 8.05289140e-059 9.78212427e-058 1.37286109e-056 2.22868258e-055
 4.18910180e-054 9.12401705e-053 2.30416959e-051 6.75018599e-050
 2.29481490e-048 9.05573718e-047 4.14878586e-045 2.20691727e-043
 1.36314441e-041 9.77672835e-040 8.14205700e-038 7.87318588e-036
 8.83953677e-034 1.15229782e-031 1.74407132e-029 3.06520599e-027
 6.25630135e-025 1.48338702e-022 4.08747805e-020 1.30979015e-017
 4.88549410e-015 2.12413014e-012 1.07870172e-009 6.41749641e-007
 4.49281690e-004 3.72714531e-001 3.70557985e+002 4.50414664e+005
 6.96339862e+008 1.50864937e+012 6.72487304e+015 8.06894942e+011
 1.69902203e+008 3.62822430e+004 7.05479089e+000 1.20421727e-003
 1.77830137e-007 2.25834206e-011 2.46124646e-015 2.30155639e-019
 1.84809339e-023 1.27586615e-027 7.58416252e-032 3.88788487e-036
 1.72151435e-040 6.59440272e-045 2.1885766

KeyboardInterrupt: 

In [ ]:
matrix_composition

In [ ]:
equilibrium(dbf, comps, ['BCC_A2', 'ETA2'], {v.N:1, v.T: 800, v.P: 1e5, v.X('CU'): 0.6})

In [ ]:
from pycalphad import calculate
import matplotlib.pyplot as plt
from pycalphad.plot.utils import phase_legend
import numpy as np

my_phases = ['ETA2', 'BCC_A2']
legend_handles, colorlist = phase_legend(my_phases)

fig = plt.figure(figsize=(9,6))
ax = fig.gca()
for name in my_phases:
    result = calculate(dbf, ['AL', 'CU', 'VA'], name, P=101325, T=300, output='GM')
    ax.scatter(result.X.sel(component='CU'), result.GM,
               marker='.', s=5, color=colorlist[name.upper()])
ax.set_xlim((0, 1))
ax.legend(handles=legend_handles, loc='center left', bbox_to_anchor=(1, 0.6))
plt.show()

In [ ]:
dbf.phases